<a href="https://colab.research.google.com/github/Birkbeck/bsc-computer-science-project-2021_22-mohammadreza490/blob/main/project_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
'''

1. Convert stems to wav. https://pypi.org/project/musdb/ 
2. For each song, split into 2 second segments. Add padding as needed
1. For each segment, create spectogram of whole mix of each song in training set via STFT
2. Create array containing spectograms of bass of each song in training set
3. Create array containing spectograms of (whole mix - bass) of each song in training set, see https://stackoverflow.com/questions/4039158/mixing-two-audio-files-together-with-python
4. Add padding to arrays such that shape is appropriate for the network.
4. Create CNN model
5. Train model
6. Apply inverse STFT / Get signal from the spectogram, see https://stackoverflow.com/questions/60377585/how-can-i-reverse-a-scipy-signal-spectrogram-to-audio-with-python
                                                        https://stackoverflow.com/questions/76447360/how-do-i-can-reconstructing-stft-to-audio
                                                        https://stackoverflow.com/questions/69387104/how-to-convert-wav-audio-file-from-mel-spectrogram
7. Test model with a bunch of songs

https://medium.com/@shameerayaseen21/u-net-advancing-image-segmentation-with-convolutional-neural-networks-1fd810f05d00
'''

'\n\n1. Convert stems to wav. https://pypi.org/project/musdb/ \n2. For each song, split into 2 second segments. Add padding as needed\n1. For each segment, create spectogram of whole mix of each song in training set via STFT\n2. Create array containing spectograms of bass of each song in training set\n3. Create array containing spectograms of (whole mix - bass) of each song in training set, see https://stackoverflow.com/questions/4039158/mixing-two-audio-files-together-with-python\n4. Add padding to arrays such that shape is appropriate for the network.\n4. Create CNN model\n5. Train model\n6. Apply inverse STFT / Get signal from the spectogram, see https://stackoverflow.com/questions/60377585/how-can-i-reverse-a-scipy-signal-spectrogram-to-audio-with-python\n                                                        https://stackoverflow.com/questions/76447360/how-do-i-can-reconstructing-stft-to-audio\n                                                        https://stackoverflow.com/ques

In [1]:
PATH_TO_SRC_FOLDER = r".\src"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install musdb
!pip install numpy
!pip install pydub
!pip install "tensorflow<2.11" #!pip install tensorflow==2.14
# !pip install tensorflow-addons==0.22.0
!pip install ipdb
!pip install museval
!pip install librosa
!pip install matplotlib


ERROR: Invalid requirement: '#!pip'


In [3]:
import os
import sys

import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import IPython.display as ipd
import musdb
import subprocess
import pydub
# import tensorflow_addons as tfa
import time
import shutil
import ipdb
import scipy
import soundfile as sf
import museval
import random
import math

from scipy.io.wavfile import write
from glob import glob

In [4]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

print(tf.config.list_physical_devices('GPU'))

tf.Tensor(267.86017, shape=(), dtype=float32)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
if PATH_TO_SRC_FOLDER not in sys.path:
  sys.path.append(PATH_TO_SRC_FOLDER) #https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab

In [6]:
from wav_handler import WavHandler
from dataset_handler import DatasetHandler
from plotter import Plotter
from model_builder import ModelBuilder
from config import INPUT_WIDTH, INPUT_HEIGHT, INPUT_CHANNELS, OUTPUT_CHANNELS

In [7]:
plotter = Plotter()
model_builder = ModelBuilder()
dataset_handler = DatasetHandler()

In [8]:
unet = model_builder.buildModel()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1040, 176,   0           []                               
                                1)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 1040, 176, 3  320         ['input_1[0][0]']                
                                2)                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 1040, 176, 3  128        ['conv2d[0][0]']                 
 alization)                     2)                                                            

In [39]:
# dataset_handler.writeTrainInputSpec()
# dataset_handler.writeTrainOutputSpec()
# dataset_handler.createBassless()

'./dataset/npy/train/output/output_padded_0.npy' already exists!
'./dataset/npy/train/output/output_padded_1.npy' already exists!
'./dataset/npy/train/output/output_padded_2.npy' already exists!
'./dataset/npy/train/output/output_padded_3.npy' already exists!
'./dataset/npy/train/output/output_padded_4.npy' already exists!
'./dataset/npy/train/output/output_padded_5.npy' already exists!
'./dataset/npy/train/output/output_padded_6.npy' already exists!
'./dataset/npy/train/output/output_padded_7.npy' already exists!
'./dataset/npy/train/output/output_padded_8.npy' already exists!
'./dataset/npy/train/output/output_padded_9.npy' already exists!
'./dataset/npy/train/output/output_padded_10.npy' already exists!
'./dataset/npy/train/output/output_padded_11.npy' already exists!
'./dataset/npy/train/output/output_padded_12.npy' already exists!
'./dataset/npy/train/output/output_padded_13.npy' already exists!
'./dataset/npy/train/output/output_padded_14.npy' already exists!
'./dataset/npy/train

In [9]:
if not os.path.exists("./model/"):
    os.mkdir("./model")

checkpoint = tf.keras.callbacks.ModelCheckpoint('./model/bass_separator_25e.h5', verbose=1, save_best_only=False, save_freq="epoch")

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, monitor='loss'), 
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        checkpoint]

train_dataset =  tf.data.Dataset.from_generator(dataset_handler.dataGenerator,
                                                       output_signature=(tf.TensorSpec(shape=(None, INPUT_WIDTH, INPUT_HEIGHT, INPUT_CHANNELS), dtype=tf.float32),
                                                                   tf.TensorSpec(shape=(None, INPUT_WIDTH, INPUT_HEIGHT, OUTPUT_CHANNELS), dtype=tf.float32)))

results = unet.fit(train_dataset, batch_size=dataset_handler.batch_size, epochs=25, callbacks=callbacks, verbose=1)

Epoch 1/25


In [29]:
unet.save('./model/bass_separator_3e.h5')

print(results.history)

{'loss': [-0.8820092678070068, -0.868935227394104, -0.8807085156440735], 'accuracy': [0.7144938111305237, 0.7122236490249634, 0.7204717993736267]}
